# Design Notebook: Calibration Dialog Fixes (RETROSPECTIVE)

**Date:** November 8-9, 2025  
**Author:** Copilot + User  
**Status:** ✅ Implemented (This is a retrospective - design was done AFTER implementation)

---

## Quick Reference
- **Problem:** Multiple calibration dialog bugs and missing features
- **Solution:** Fixed shift-click, bounds, offset loading, cancel behavior, added orientation buttons
- **Impact:** display_control.py, SerialProtocol.cpp, DisplayManager.cpp/h
- **Complexity:** High

---

**NOTE:** This notebook was created AFTER implementation as a learning exercise. Going forward, notebooks like this MUST be created BEFORE any code is written.

# 1. Problem Statement

## What were we trying to solve?

User reported 7 issues with the calibration dialog:

1. Image transfer speed (minor improvement only)
2. Shift-click width/height adjustment not working correctly
3. Bounds calculations were incorrect
4. Cancel button didn't restore config file values
5. Missing orientation rotation buttons
6. Current offset values not loading when dialog opens
7. Spinbox values not updating after adjustments

## User Requirements

- **Shift+click behavior**: Should adjust width/height symmetrically
  - Top+: Increase height (move top up AND bottom down)
  - Left+: Increase width (move left left AND right right)
- **Bounds**: Based on display dimensions, not hardcoded ±50
- **Cancel**: Should restore values from .config file
- **Orientation**: Need buttons for all 4 orientations
- **State persistence**: Values should load from firmware on dialog open

## Pain Points

- Inefficient iteration: Issues discovered during testing, not design
- Multiple rounds of fixes needed
- Confusion about bounds calculation logic
- Firmware/Python state synchronization issues

## Success Criteria

- [x] Shift-click adjusts width/height correctly
- [x] Bounds calculated from display dimensions
- [x] Current offsets load on dialog open
- [x] Cancel restores .config values via UPDATE_CONFIG
- [x] Orientation buttons change display rotation
- [x] All spinboxes update after adjustments

# 2. Current State Analysis (What should have been known)

## What existed

- `display_control.py`: Python GUI with calibration dialog
- `SerialProtocol.cpp`: Firmware command handler
- `DisplayManager.cpp/h`: Display abstraction layer
- `.config` files: TOML configuration storage

## Files that needed changes

| File | Current State | Changes Needed |
|------|--------------|----------------|
| `display_control.py` | Shift-click logic wrong, hardcoded bounds, late offset loading | Fix logic, calculate bounds, load offsets early, add orientation UI |
| `SerialProtocol.cpp` | No ORIENTATION command | Add CMD:ORIENTATION handler |
| `DisplayManager.h` | No orientation setter exposed | Expose setRotation via API |

## Current Limitations

1. **Shift-click**: Both edges moved same direction (resize instead of translate)
2. **Bounds**: Hardcoded ±50 instead of calculated from display dimensions
3. **Loading order**: Offsets loaded AFTER spinboxes created (values not shown)
4. **Cancel behavior**: Only reset to session start, not .config file
5. **Orientation**: No UI controls, had to restart firmware
6. **State sync**: Python and firmware could get out of sync

## Dependencies

- Python: tkinter, toml, pyserial
- Firmware: Adafruit_ST7735, Adafruit_GFX

# 3. Technical Design (What SHOULD have been designed)

## Architecture Overview

```
┌─────────────────┐         Serial Commands          ┌──────────────────┐
│  Python GUI     │────────────────────────────────>│  Arduino Due     │
│ display_control │<────────────────────────────────│  SerialProtocol  │
│                 │         Responses/Status         │                  │
└─────────────────┘                                  └──────────────────┘
        │                                                     │
        │ Read/Write                                         │ Controls
        ▼                                                     ▼
┌─────────────────┐                                  ┌──────────────────┐
│  .config files  │                                  │  DisplayManager  │
│     (TOML)      │                                  │   ST7735 TFT     │
└─────────────────┘                                  └──────────────────┘
```

## Key Components

### 1. Shift-Click Logic
**Correct behavior:**
- Shift+Top+: `top -= 1, bottom -= 1` (both move same direction = height increase)
- Shift+Left+: `left -= 1, right -= 1` (both move same direction = width increase)

**Wrong behavior (original):**
- Shift+Top+: `top -= 1, bottom += 1` (opposite directions = move not resize)

### 2. Bounds Calculation
**Correct:**
```python
# Get from INFO command
display_width, display_height = 160, 128
usable_x, usable_y = 1, 1
usable_width, usable_height = 158, 126

# Calculate max adjustments
max_top_adjust = usable_y + 10  # Can go 10px beyond top edge
max_bottom_adjust = (display_height - (usable_y + usable_height)) + 10
max_left_adjust = usable_x + 10
max_right_adjust = (display_width - (usable_x + usable_width)) + 10
```

**Wrong (original):**
```python
max_top_adjust = center_y - 10  # Hardcoded, doesn't account for display geometry
```

### 3. State Loading Order
**Correct:**
1. Send INFO command
2. Parse all values including offsets
3. Create spinboxes with loaded values
4. Store original values for cancel

**Wrong (original):**
1. Create spinboxes with 0
2. Send INFO later
3. Try to update already-created spinboxes

### 4. Cancel Behavior
**Correct:**
```python
def cancel_and_exit():
    config = toml.load(f"{display_name}.config")
    left, right, top, bottom = config['calibration']['left/right/top/bottom']
    center_x, center_y = config['calibration']['center']
    send_command(f"UPDATE_CONFIG:{left},{right},{top},{bottom},{center_x},{center_y}")
```

**Wrong (original):**
```python
def cancel_and_exit():
    for side in ['TOP', 'BOTTOM', 'LEFT', 'RIGHT']:
        send_command(f"ADJUST_{side}:{original_offsets[side]}")  # Only session state
```

# 4. Implementation Plan (Actual execution vs what should have been planned)

## What SHOULD have been the task breakdown

### Task 1: Fix Shift-Click Logic
- **Files:** `display_control.py` (adjust_top/bottom/left/right functions)
- **Change:** Both edges move same direction (not opposite)
- **Complexity:** Low
- **Time:** 15 minutes
- **Status:** ✅ Complete

### Task 2: Fix Bounds Calculations  
- **Files:** `display_control.py` (calibrate_display function)
- **Change:** Parse display dimensions from INFO, calculate bounds dynamically
- **Complexity:** Medium
- **Time:** 30 minutes
- **Status:** ✅ Complete

### Task 3: Load Offsets Early
- **Files:** `display_control.py`
- **Change:** Move offset loading before spinbox creation
- **Complexity:** Low
- **Time:** 15 minutes  
- **Status:** ✅ Complete

### Task 4: Fix Cancel Behavior
- **Files:** `display_control.py`
- **Change:** Read .config file, send UPDATE_CONFIG with those values
- **Dependencies:** Requires `import toml`
- **Complexity:** Medium
- **Time:** 20 minutes
- **Status:** ✅ Complete

### Task 5: Add Orientation Buttons
- **Files:** `display_control.py`, `SerialProtocol.cpp`
- **Change:** Add UI buttons, implement CMD:ORIENTATION handler
- **Complexity:** Medium
- **Time:** 30 minutes
- **Status:** ✅ Complete

### Task 6: Add Image Cropping
- **Files:** `DisplayManager.h/cpp`, `SerialProtocol.cpp`
- **Change:** Add isWithinFrameBounds(), use in pixel drawing
- **Complexity:** Medium  
- **Time:** 20 minutes
- **Status:** ✅ Complete

## What ACTUALLY happened

- Started coding immediately without planning
- Discovered issues during testing
- Fixed issues one by one as reported
- Multiple firmware upload cycles
- Inefficient back-and-forth iteration

## Total Time

- **Planned (if designed first):** ~2.5 hours
- **Actual (without design):** ~4-5 hours + multiple test/fix cycles

# 5. Lessons Learned

## What Went Wrong

1. **No upfront design**: Started coding without understanding full scope
2. **Reactive fixes**: Fixed issues as they were discovered, not proactively
3. **Multiple iterations**: Each issue required separate fix + test + upload cycle
4. **State confusion**: Didn't map out Python/firmware state synchronization
5. **Bounds logic unclear**: Math errors because requirements weren't written down

## What Caused Inefficiency

| Issue | Root Cause | Prevention |
|-------|-----------|------------|
| Shift-click wrong | Didn't write out the math first | Pseudocode in design notebook |
| Bounds wrong | Didn't understand display geometry | Diagram in notebook |
| Late offset loading | Didn't trace code flow | Flow diagram in notebook |
| Multiple uploads | Fixed issues one at a time | Group related fixes together |

## Cost of No Design

- **Extra time**: ~2 hours wasted
- **Mental load**: Tracking multiple issues simultaneously
- **Risk**: Could have introduced bugs in other areas
- **Documentation**: No clear record of why decisions were made

## What Design Notebook Would Have Prevented

If we had created a design notebook FIRST:

1. **Math errors caught early**: Writing out shift-click logic would have revealed the error
2. **Bounds calculated correctly**: Diagram would show proper geometry
3. **State sync clear**: Architecture diagram would show data flow
4. **All fixes grouped**: Would have uploaded firmware once, not 3-4 times
5. **Test cases defined**: Would have tested all scenarios before declaring done

## Key Insight

> **Time spent planning = Time saved implementing**
> 
> 30 minutes designing > 2 hours debugging